In [12]:
%%capture
!pip install ipython-autotime

%load_ext autotime

time: 2.63 s (started: 2021-08-29 21:15:55 +00:00)


# Instala a lib Reranker

In [13]:
%cd /content/drive/MyDrive/__TCC__Reranker/Reranker

/content/drive/MyDrive/__TCC__Reranker/Reranker
time: 8.23 ms (started: 2021-08-29 21:15:57 +00:00)


In [14]:
%%capture
%pip install -e .

time: 4.53 s (started: 2021-08-29 21:15:57 +00:00)


In [15]:
!export HF_DATASETS_CACHE="/content/drive/My Drive/__TCC__Reranker"

time: 116 ms (started: 2021-08-29 21:16:02 +00:00)


In [16]:
import nltk
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)

True

time: 31.9 ms (started: 2021-08-29 21:16:02 +00:00)


# Preprocessa apenas 1 arquivo hdct

In [24]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/scripts/1_preprocess_lsa_summarization.py \
    --tokenizer_name bert-base-uncased \
    --rank_file 	/content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/01-hdct-marco-train/000.txt \
    --json_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/training-files \
    --n_sample 10 \
    --sample_from_top 100 \
    --random \
    --truncate 512 \
    --qrel /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-doctrain-qrels.tsv.gz \
    --query_collection /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-doctrain-queries.tsv \
    --doc_collection /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/02-msmarco-files/msmarco-docs.tsv

========== LSA SUMMARIZATION ==========
0 queries not judged and skipped
Using custom data configuration default-336b90812e3158ed
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-336b90812e3158ed/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
Using custom data configuration default-81f490248c7c9f05
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-81f490248c7c9f05/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff)
  0% 0/2000 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors
  0% 1/2000 [00:01<1:01:26,  1.84s/it]/content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/scripts/lsa_summarizer.py:72: UserWarning: Number of words (1709) is lower than number of sentences (1746). LSA algorithm may not work properly.
  warn(message % (words_count, sentenc

# Realiza treino

In [30]:
!python -m torch.distributed.launch \
  --nproc_per_node 1 /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py \
  --output_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/checkpoints \
  --model_name_or_path  bert-base-uncased \
  --do_train \
  --save_steps 2000 \
  --train_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/training-files/ \
  --max_len 512 \
  --fp16 \
  --per_device_train_batch_size 1 \
  --train_group_size 8 \
  --gradient_accumulation_steps 1 \
  --per_device_eval_batch_size 64 \
  --warmup_ratio 0.1 \
  --weight_decay 0.01 \
  --learning_rate 1e-5 \
  --num_train_epochs 2 \
  --overwrite_output_dir \
  --dataloader_num_workers 8 \

/usr/local/lib/python3.7/dist-packages/torch/distributed/launch.py:164: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  "The module torch.distributed.launch is deprecated "
The module torch.distributed.launch is deprecated and going to be removed in future.Migrate to torch.distributed.run
 Please read local_rank from `os.environ('LOCAL_RANK')` instead.
INFO:torch.distributed.launcher.api:Starting elastic_operator with launch configs:
  entrypoint       : /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py
  min_nodes        : 1
  max_nodes        : 1
  nproc_per_node   : 1
  run_id           : none
  rdzv_backend     : static
  rdzv_endpoint    : 127.0.0.1:29500
  rdzv_configs     : {'rank': 0, 'timeout': 900}
  max_restarts     : 3
  monitor_interval : 5
  log_dir          : None
  metrics_cfg      : {}

INFO:torch.distributed.elastic.agent.server.local_elastic_agent:log directory set to: /tmp/torchelastic_1vuvkjy0/none_s7tmuxi0
I

# Realiza Inference

In [31]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/2_run_marco.py \
  --model_name_or_path /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/checkpoints \
  --output_dir /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/eval \
  --tokenizer_name bert-base-uncased \
  --do_predict \
  --max_len 512 \
  --fp16 \
  --per_device_eval_batch_size 64 \
  --dataloader_num_workers 8 \
  --pred_path /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/04-inference_files/all.json \
  --pred_id_file  /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/04-inference_files/ids.tsv \
  --rank_score_path /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/eval/score.txt



AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAa

=========== CONFIGURANDO AS COISAS ===========

===========  GETTING DATASET ===========

===========  INITIALIZING TRAINER ===========
Using amp fp16 backend

===========  PREDICTION ===========
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-356e9d9d9447f953/0.0.0/45636811569ec4a6630521c18235dfbbab83b7ab572e3393c5ba68ccabe98264. Subsequent calls will reuse this data.
***** Running Prediction *****
  Num examples = 519300
  Batch size = 64
100% 8114/8115 [2:28:14<00:01,  1.10s/it]
===========  TERMINOU ===========
100% 8115/8115 [2:28:47<00:00,  1.10s/it]
time: 2h 29min 56s (started: 2021-08-30 01:23:08 +00:00)


# Converte para MSmarco datafile

In [32]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/4_score_to_marco.py \
 --score_file /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/eval/score.txt

time: 1.43 s (started: 2021-08-30 03:53:04 +00:00)


# Avalia inference

In [33]:
!python /content/drive/MyDrive/__TCC__Reranker/Reranker/common_scripts/6_msmarco_eval_2.py \
 /content/drive/MyDrive/__TCC__Reranker/Reranker/run_lsa_summarization/data/eval/score.txt.marco \
  /content/drive/MyDrive/__TCC__Reranker/Reranker/run_msmarco/data/eval/final_score/dev_ground_truth.tsv \
  /content/drive/MyDrive/__TCC__Reranker/Reranker/common_data/eval/exclude

398 excluded qids loaded
Quantity of Documents ranked for each query is as expected. Evaluating
#####################
MRR @100: 0.3477914349579377
QueriesRanked: 5193
#####################
time: 921 ms (started: 2021-08-30 03:53:06 +00:00)
